# Assignment Data Structure and Solution
#### We can use this Jupyter Notebook to analysis the data structure of different file and variables to help us to solve these three model construction and analysis.

# First Class(CommandLine)
<big>**Calss Function**</big>:
   1. Judge the command line to accord with the stationary Formate.
   2. Print Help Inforamtion if the command formate is wrong.
   3. Default:<br>
      1)'index_nostoplist_nostemming.txt' / queries_nostoplist_nostemming.txt'. <br>
      2) For the First dataset(index?), it's the key vocabularly in the text. <br>
      3) For the second dataset(queries?), it's the summary of all the search queris. <br>
       
**Example Command Formate**:
```
> python ir_engine.py -o result.txt
```

In [1]:
#==============================================================================
# Importing
import sys, getopt, re
import time
#==============================================================================
# Command line processing
class CommandLine:
    def __init__(self):
        opts, args = getopt.getopt(sys.argv[1:], 'hspw:o:')
        opts = dict(opts)
        self.exit = True

        if '-h' in opts:
            self.printHelp()
            return

        if len(args) > 0:
            print("*** ERROR: no arg files - only options! ***", file=sys.stderr)
            self.printHelp()
            return

        if '-w' in opts:
            if opts['-w'] in ('binary', 'tf', 'tfidf'):
                self.termWeighting = opts['-w']
            else:
                warning = (
                    "*** ERROR: term weighting label (opt: -w LABEL)! ***\n"
                    "    -- value (%s) not recognised!\n"
                    "    -- must be one of: binary / tf / tfidf"
                    )  % (opts['-w'])
                print(warning, file=sys.stderr)
                self.printHelp()
                return
        else:
            self.termWeighting = 'binary'

        if '-o' in opts:
            self.outfile = opts['-o']
        else:
            print("*** ERROR: must specify output file (opt: -o FILE) ***",
                  file=sys.stderr)
            self.printHelp()
            return

        if '-s' in opts and '-p' in opts:
            self.indexFile   = 'index_withstoplist_withstemming.txt'
            self.queriesFile = 'queries_withstoplist_withstemming.txt'
        elif '-s' in opts:
            self.indexFile   = 'index_withstoplist_nostemming.txt'
            self.queriesFile = 'queries_withstoplist_nostemming.txt'
        elif '-p' in opts:
            self.indexFile   = 'index_nostoplist_withstemming.txt'
            self.queriesFile = 'queries_nostoplist_withstemming.txt'
        else:
            self.indexFile   = 'index_nostoplist_nostemming.txt'
            self.queriesFile = 'queries_nostoplist_nostemming.txt'
            
        self.exit = False

    def printHelp(self):
        progname = sys.argv[0]
        progname = progname.split('/')[-1] # strip off extended path
        help = __doc__.replace('<PROGNAME>', progname, 1)
        print(help, file=sys.stderr)


# Second Class(IndexLoader)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 dimensions.

<big>**Data Structure(Python Dictionary)**</big>:<br>
   1. {(term name)->{(doc_id)-> Count}}

In [2]:
#==============================================================================
# Load (precomputed) Index File for (preprocessed) Document Collection
class IndexLoader:
    def __init__(self, indexFile):
        self.index = {}
        docidCountRE = re.compile('(\d+):(\d+)')
        f = open(indexFile, 'r')
        for line in f:
            term = line.split(' ', 1)[0]
            self.index[term] = {}
            for (docid, count) in docidCountRE.findall(line):
                docid = int(docid)
                self.index[term][docid] = int(count)
    def getIndex(self):
        return self.index

# Third Class(Queries)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 or more dimensions.
   3. This class is to extract the queries and get the same dataset formate(python dictionary) as IndexLoader

<big>**Data Structure(Dictionary)**</big>:<br>
   1. {(qid name)->{(qid name)-> count_num}}

In [3]:
class Queries:
    def __init__(self, queriesFile):
        self.qStore = {}
        termCountRE = re.compile('(\w+):(\d+)')
        f = open(queriesFile, 'r')
        for line in f:
            qid = int(line.split(' ', 1)[0])
            self.qStore[qid] = {}
            for (term, count) in termCountRE.findall(line):
                self.qStore[qid][term] = int(count)
    
    def getQuery(self, qid):
        if qid in self.qStore:
            return self.qStore[qid]
        else:
            print("*** ERROR: unknown query identifier (\"%s\") ***" % qid, file=sys.stderr)
            if type(qid) == type(''):
                print('WARNING: query identifiers should be of type: integer', file=sys.stderr)
                print('         -- your query identifier is of type: string', file=sys.stderr)
            print(' -- program exiting', file=sys.stderr)
    
    def qids(self):
        return sorted(self.qStore)

# Forth Class(ResultStore)
<big>**Calss Function**</big>:<br>
<big>**Data Structure(Dictionary)**</big>:<br>

In [4]:
class ResultStore:
    def __init__(self, outfile):
        self.outfile = outfile
        self.results = []

    def store(self, qid, docids):
        if len(docids) > 10:
            docids = docids[:10]
        self.results.append((qid, docids))

    def output(self):
        with open(self.outfile, 'w') as out:
            for (qid, docids) in self.results:
                for docid in docids:
                    print(qid, docid, file=out)

# Target Class(Retrieve)

In [90]:
# TODO: Finish the Assignment Part
import math
import operator
import collections
class Retrieve:
    # Define all the usable class variables
    index = None
    termWeighting = None
    doc_len = dict()  # The dictionary to record all the documents' length
    tr_candidate = dict()  # Retrieval all the documents 
    tr_candidate_all = dict() # for every doc_id and count
    tr_candidate_id = list()  # Retrial all the documents ID and store for use binary model candidate
    tr_query_count = list()  # To store the query count/frequency

    # Create new Retrieve object storing index and termWeighting scheme
    def __init__(self, index, termWeighting):
        self.index = index
        self.termWeighting = termWeighting
        # get the length of the each documents and
        self.doc_length()

    # Firstly we should calculate the documents length by number of arisen vocabularly
    def doc_length(self):
        # Extract all the items from the index file and sum the arisen number
        tempor_list = self.index.items()
        for term,combination in tempor_list:
            for doc_id,show_count in combination.items():
                if doc_id in self.doc_len:
                    self.doc_len[doc_id] = self.doc_len[doc_id] + int(show_count)
                else:
                    self.doc_len[doc_id] = int(show_count)

    # Method performing retrieval for specified query
    def forQuery(self, query):
        # Compare the index and query
        self.getCandidate(index, query) 
        # Firstly we should judge the function the program operate
        if self.termWeighting is 'binary':
            output = self.binary_model(query)
        elif self.termWeighting is 'tf':
            output = self.tf_model(query)
        elif self.termWeighting is 'tfidf':
            output = self.tfidf_model(query)      
        return output
        

    # Extract the query from the file and also generate a new corresponding dictionary
    # We delete all the irreverent values and integrate the dictionary into a list(Count all the count)
    def getCandidate(self, index, query):
        # Get the two list to store the term of query and number of key vocabularly
        queryIterm = list(query.keys())
        qtermCount = list(query.values())
        for each_iterm in set(queryIterm):            
            if each_iterm in index:   # If the target document includes the query key we should put the id into a document
                self.tr_candidate[each_iterm] = index[each_iterm] # get all the document data formate: doc_id:count
                for doc_id,count in index[each_iterm].items():
                    if doc_id in self.tr_candidate_all:
                        self.tr_candidate_all[doc_id][each_iterm] = count
                    else:
                        self.tr_candidate_all[doc_id] = {}
                id_list = list(index[each_iterm].keys())
                self.tr_candidate_id.extend(id_list)
    
    
    def binary_model(self, query):
        Results = dict()
        score = 0
        # For binary we just care about the occurance in query and doc or not, every occurance we regard as the same weight
        # So we just care about the occurance of index(doc) and treat every occurance as once
        all_lists = collections.Counter(self.tr_candidate_id)
        for doc_id, count in all_lists.items():
            # Count is all the value for the molecule of formula(sum)
            score = count / (math.sqrt(self.doc_len[doc_id]))
            Results[doc_id] = score       
        topscore = [doc_id for doc_id,value in sorted(Results.items(), key = operator.itemgetter(1), reverse=True)]
        return topscore
            
                     
    def tf_model(self, query):
        Results = dict()
        # Calculate all the value from the self.tr_documentid and convert it into the set to improve the efficiency
        # We have built the tr_candidate_id to store all the show index file and number is the occurance of query     
        # We need to calculate the number of occurance of index in set of query
        tf_candidate = set(self.tr_candidate_id)
        score = 0        
        for doc_id in tf_candidate:
            for each_term,each_value in (query.items()):
                if each_term in self.tr_candidate_all[doc_id]:
                    score = score + self.tr_candidate_all[doc_id][each_term] * each_value
                else:
                    continue
            result = score / math.sqrt(self.doc_len[doc_id])
            Results[doc_id] = result
            score = 0
        # return value is a list value
        topscore = [doc_id for doc_id,value in sorted(Results.items(), key = operator.itemgetter(1), reverse = True)]
        return topscore
    
    
    def tfidf_model(self, query):
        # First we need to calculate value of Inverse document frequency(IDF) value
        IDFValue = {}
        for term,collection in self.index.items():
            IDFValue[term] = math.log(len(self.doc_len)/len(collection))
        
        # Next we should create two new data-structure(dic()) to store the IDF and score
        # For the IDF value for each doc_id
        Results = dict()
        doc_idf = dict()
        for term,qterm_frequency in query.items():
            if term in self.tr_candidate:
                for doc_id, count in self.tr_candidate[term].items():
                    frequency = qterm_frequency * count
                    if doc_id in doc_idf:
                        doc_idf[doc_id] += frequency * IDFValue[term]
                    else:
                        doc_idf[doc_id]  = frequency * IDFValue[term]
        for doc_id,value in doc_idf.items():
            Results[doc_id] = value/math.sqrt(self.doc_len[doc_id])       
        topscore = [doc_id for doc_id,value in sorted(Results.items(), key = operator.itemgetter(1), reverse = True)]
        return topscore            


<big>**Below Code is the Formate of Command(Please customize with on the basis of your requirements)**</big><br><br>

**How to generate command line As you wish**<br>

`sys.argv` is the python reserved variables and applied to `ArgumentParser.parse_args(args=None, namespace=None)` to be the one of the parameters. This is a list of strings, each of which is obviously derived from command line arguments, separated by spaces. We can redefine this list by below command.

We should delete the key variables like `python` and put the remains into the list.

In [91]:
# Example Command Formate:
#  > python ir_engine.py -o result.txt
sys.argv = ['ir_engine.py', '-w', 'tfidf', '-o', 'result1.txt']

In [92]:
config = CommandLine()
index = IndexLoader(config.indexFile).getIndex()

In [93]:
retrieve = Retrieve(index, config.termWeighting)
queries = Queries(config.queriesFile)
allResults = ResultStore(config.outfile)

In [94]:
for qid in queries.qids():
    query = queries.getQuery(qid)
    results = retrieve.forQuery(query)
    print(results)
    allResults.store(qid, results)

allResults.output()

[1938, 2319, 1591, 2379, 1680, 1827, 2371, 1523, 1033, 1605, 1071, 1519, 2629, 1657, 2380, 1844, 971, 2344, 2218, 1264, 1908, 1440, 2740, 1410, 1161, 3048, 1544, 1572, 1885, 1393, 3087, 2541, 2439, 3127, 2424, 1542, 2188, 2948, 1173, 3112, 3068, 3023, 1571, 1304, 2219, 1464, 1752, 1226, 1315, 2378, 2003, 1506, 2398, 1937, 2535, 2632, 2092, 1647, 585, 1518, 1195, 2153, 2947, 414, 2054, 2841, 1236, 2542, 2357, 637, 1697, 1179, 252, 1626, 2796, 2501, 1665, 2951, 2317, 2560, 1698, 202, 2984, 2500, 2764, 2126, 2311, 1685, 1163, 1108, 3069, 2470, 2105, 972, 1168, 1960, 1238, 1170, 1482, 1956, 1462, 1472, 1719, 2917, 1412, 3137, 1247, 2955, 2950, 2597, 2080, 1391, 1682, 1034, 1505, 2681, 2151, 1534, 2065, 2526, 2811, 1902, 2878, 2868, 1225, 1487, 1959, 2169, 3025, 3012, 2288, 1347, 2622, 1533, 1069, 2370, 1280, 2358, 1589, 1755, 1642, 2274, 2201, 1769, 1829, 1046, 3082, 3089, 397, 3153, 2920, 2536, 2340, 696, 698, 1811, 1295, 98, 3091, 3131, 2627, 2959, 284, 2064, 2390, 3174, 2372, 294, 1853,

[3047, 1327, 3013, 2826, 533, 1588, 1144, 1741, 2283, 1108, 1727, 1543, 1686, 3059, 1665, 3015, 2931, 2125, 1381, 2233, 2721, 2188, 3077, 670, 2893, 2051, 3076, 1847, 2421, 2554, 1261, 2370, 1366, 1856, 1721, 2020, 1891, 1961, 1664, 2916, 2195, 2628, 724, 2622, 849, 2211, 1388, 2829, 2003, 464, 1423, 2092, 2911, 2302, 696, 3160, 1601, 1098, 1652, 2402, 1671, 2470, 78, 3157, 1194, 2842, 1534, 3112, 972, 616, 2002, 1350, 2054, 1680, 2390, 3200, 2697, 2753, 531, 1645, 1827, 1415, 2216, 2534, 2264, 2033, 2289, 1497, 2113, 1638, 2645, 321, 615, 1379, 1700, 2142, 1233, 2535, 1929, 1359, 695, 3153, 2786, 2479, 1695, 2396, 1928, 2702, 2186, 1307, 2274, 2606, 1206, 2163, 1571, 1936, 1066, 2999, 2572, 2082, 1383, 1236, 2889, 2367, 1829, 2130, 2217, 3090, 2939, 3020, 3028, 730, 2971, 1325, 2922, 1641, 2313, 1195, 2993, 1805, 2568, 1433, 104, 497, 605, 2846, 796, 2883, 2096, 2181, 2841, 1003, 2060, 1901, 2198, 2254, 2855, 2998, 1455, 2252, 1625, 1135, 2940, 2886, 1807, 2751, 1806, 1844, 2851, 855,

[3077, 3140, 1106, 1137, 1132, 196, 2700, 2219, 437, 2922, 2610, 2931, 2470, 1705, 1469, 1931, 2864, 1470, 1, 209, 1856, 1695, 176, 1835, 2113, 1665, 3099, 2802, 3036, 2344, 689, 2020, 2645, 1727, 2850, 1391, 1844, 3080, 2886, 796, 2181, 71, 1194, 406, 527, 2876, 980, 2705, 2097, 1155, 1934, 2937, 1029, 1834, 3002, 2178, 2883, 1278, 2943, 2480, 3008, 2506, 616, 464, 1170, 1458, 2920, 1003, 2971, 259, 2396, 1811, 65, 2390, 2484, 2175, 1162, 2106, 3031, 1715, 2061, 1766, 1323, 2842, 3160, 3027, 1961, 1423, 2378, 1638, 2236, 2716, 2083, 3049, 1805, 1135, 1012, 2905, 3078, 1188, 3050, 2746, 1771, 2201, 3150, 2451, 3015, 1534, 1327, 1764, 3148, 2898, 1885, 2739, 2202, 1788, 2167, 1854, 3014, 2437, 1543, 2718, 2002, 2164, 2682, 2494, 1572, 2092, 2197, 2060, 2341, 1852, 2568, 2402, 2851, 1677, 3154, 1591, 1433, 2233, 2324, 2053, 3041, 1366, 248, 1196, 2342, 3000, 1614, 724, 1482, 2215, 875, 615, 1630, 2227, 1605, 1421, 972, 1307, 1997, 1046, 2846, 1527, 2186, 2925, 2929, 1974, 2264, 2650, 253